In [184]:
import tensorflow as tf
import numpy as np

In [185]:
tf.compat.v1.disable_eager_execution()

In [186]:
class TransformerEncoder:
    def __init__(self, vocab_size, seq_len, emb_dim, linear_layers_hidden_dim, ffn_hiiden_dim):

        self.vocab_size = vocab_size
        self.seq_len = seq_len
        self.emb_dim = emb_dim

        self.linear_layers_hidden_dim = linear_layers_hidden_dim
        self.ffn_hiiden_dim = ffn_hiiden_dim

 
    def get_sin_output(self, pos):
        t = tf.multiply(tf.constant(2, dtype=tf.float32), tf.range(0, self.emb_dim, dtype=tf.float32))
        t1 = tf.pow(tf.constant(10000, dtype=tf.float32), tf.divide(t, self.emb_dim))
        position_tensor = tf.fill([1, self.emb_dim], pos)
        sin_fun_input = tf.divide(position_tensor, t1)
        output = tf.sin(sin_fun_input)
        return output
    
    def get_cos_output(self, pos):
        t = tf.multiply(tf.constant(2, dtype=tf.float32), tf.range(0, self.emb_dim, dtype=tf.float32))
        t1 = tf.pow(tf.constant(10000, dtype=tf.float32), tf.divide(t, self.emb_dim))
        position_tensor = tf.fill([1, self.emb_dim], pos)
        sin_fun_input = tf.divide(position_tensor, t1)
        output = tf.cos(sin_fun_input)
        return output
    
    def call(self, X):

        with tf.compat.v1.Session() as attention_head:
            
            # X is data such as [[hello, world],[amazing, code]]. It is in the format of int32       
            # first embedding layer
            # start of embedding graph
            inputs = tf.compat.v1.placeholder(dtype=tf.float32, shape=(1, self.vocab_size), name='InputPlaceholder')
            input_pos = tf.compat.v1.placeholder_with_default(0.0, shape=(), name='input_pos')
            w1 = tf.Variable(tf.random.normal(shape=(self.vocab_size, self.emb_dim), dtype=tf.float32), name='w1')
            b1 = tf.Variable(tf.constant(0.1, shape=(1, self.emb_dim), dtype=tf.float32), name='b1')
            y1 = tf.add(tf.matmul(inputs, w1), b1)
            # end of embedding graph
            
            #positional embeddings 
            @tf.function
            def conditoinal(pos):
                if tf.equal(tf.experimental.numpy.mod(pos,2) ,0):
                    return self.get_sin_output(pos)
                return self.get_cos_output(pos)
            
            positional_out = tf.add(conditoinal(input_pos), y1)
            # end of positional output
            
            
            #value linear layer
            inputs_v = tf.compat.v1.placeholder(dtype=tf.float32, shape=(self.seq_len, self.emb_dim), name='linear_layer_graph_placeholder_v')
            w1_v = tf.Variable(tf.random.normal(shape=(self.emb_dim, self.linear_layers_hidden_dim), dtype=tf.float32), name='w1_v')
            b1_v = tf.Variable(tf.constant(0.1, shape=(1, self.linear_layers_hidden_dim), dtype=tf.float32), name='b1_v')
            y1_v = tf.add(tf.matmul(inputs_v, w1_v), b1_v)
            
            #key linear layer
            inputs_k = tf.compat.v1.placeholder(dtype=tf.float32, shape=(self.seq_len, self.emb_dim), name='linear_layer_graph_placeholder_k')
            w1_k = tf.Variable(tf.random.normal(shape=(self.emb_dim, self.linear_layers_hidden_dim), dtype=tf.float32), name='w1_k')
            b1_k = tf.Variable(tf.constant(0.1, shape=(1, self.linear_layers_hidden_dim), dtype=tf.float32), name='b1_k')
            y1_k = tf.add(tf.matmul(inputs_k, w1_k), b1_k)
            
            #query linear layer
            inputs_q = tf.compat.v1.placeholder(dtype=tf.float32, shape=(self.seq_len, self.emb_dim), name='linear_layer_graph_placeholder_q')
            w1_q = tf.Variable(tf.random.normal(shape=(self.emb_dim, self.linear_layers_hidden_dim), dtype=tf.float32), name='w1_q')
            b1_q = tf.Variable(tf.constant(0.1, shape=(1, self.linear_layers_hidden_dim), dtype=tf.float32), name='b1_q')
            y1_q = tf.add(tf.matmul(inputs_q, w1_q), b1_q)
            
            
            transposed_key_mat = tf.transpose(y1_k, name='key_transpose_op')
            attention_filter = tf.divide(tf.matmul(y1_q, transposed_key_mat), tf.sqrt(tf.constant(self.emb_dim, dtype=tf.float32)), name='attention_filter')
            softmaxed_attention_filter = tf.nn.softmax(attention_filter, name='softmax_of_attention_filter')
            final_output_of_one_head = tf.matmul(softmaxed_attention_filter, y1_v, name='final_matmul')
            
            # final linear layer
            inputs_final = tf.compat.v1.placeholder(dtype=tf.float32, shape=(self.seq_len, self.linear_layers_hidden_dim), name='linear_layer_graph_placeholder_final')
            w1_final = tf.Variable(tf.random.normal(shape=(self.linear_layers_hidden_dim, self.emb_dim), dtype=tf.float32), name='w1_final')
            b1_final = tf.Variable(tf.constant(0.1, shape=(1, self.emb_dim), dtype=tf.float32), name='b1_final')
            y1_final = tf.add(tf.matmul(inputs_final, w1_final), b1_final)
            
            #here multhead attention finish
            
            # now onto add and norm layer
            inputs_for_norm = tf.compat.v1.placeholder(dtype=tf.float32, shape=(self.seq_len, self.emb_dim), name='InputPlaceholderNorm')
            add_op = tf.add(inputs_for_norm,y1_final)
            meaned_output = tf.reduce_mean(y1_final, axis=1)
            std_output = tf.math.reduce_std(y1_final, axis=0)
            x0 = tf.transpose(tf.subtract(tf.transpose(add_op), meaned_output))
            divisor = tf.sqrt(tf.add(std_output, tf.constant(0.001, dtype=tf.float32)))
            res = tf.divide(x0, divisor)
            # finsish add and norm layer
            
            #feed forward network
            inputs_for_ffn = tf.compat.v1.placeholder(dtype=tf.float32, shape=(self.seq_len, self.emb_dim), name='InputPlaceholderLinear')
            w1_ffn = tf.Variable(tf.random.normal(shape=(self.emb_dim, self.ffn_hiiden_dim), dtype=tf.float32), name='w1_ffn')
            b1_ffn = tf.Variable(tf.constant(0.1, shape=(1, self.ffn_hiiden_dim), dtype=tf.float32), name='b1_ffn')
            y1_ffn = tf.nn.relu(tf.add(tf.matmul(inputs_for_ffn, w1_ffn), b1_ffn))
            w2_ffn = tf.Variable(tf.random.normal(shape=(self.ffn_hiiden_dim, self.emb_dim), dtype=tf.float32), name='w2_ffn')
            b2_ffn = tf.Variable(tf.constant(0.1, shape=(1, self.emb_dim), dtype=tf.float32), name='b2_ffn')
            y2_ffn = tf.add(tf.matmul(y1_ffn, w2_ffn), b2_ffn)
            #end of feed forward neural network
            
            #now onto final add and norm layer
            inputs_for_norm_final = tf.compat.v1.placeholder(dtype=tf.float32, shape=(self.seq_len, self.emb_dim), name='InputPlaceholderLinear')
            add_op_final = tf.add(inputs_for_norm_final,y2_ffn)
            meaned_output_final = tf.reduce_mean(y2_ffn, axis=1)
            std_output_final = tf.math.reduce_std(y2_ffn, axis=0)
            x0_final = tf.transpose(tf.subtract(tf.transpose(add_op_final), meaned_output_final))
            divisor_final = tf.sqrt(tf.add(std_output_final, tf.constant(0.001, dtype=tf.float32)))
            res_final = tf.divide(x0_final, divisor_final)
            #driver code
            
            init_op = tf.compat.v1.global_variables_initializer()
            attention_head.run(init_op)
            
            sentences = []
            final_output = []
            
            for sent in X:
                one_hot_encoded = tf.one_hot(sent, dtype=tf.float32, depth=self.vocab_size)
                sent_vector = []
                for pos,word in enumerate(tf.unstack(one_hot_encoded)):
                    word = tf.reshape(word, shape=(1,len(vocab))).eval(session=attention_head)
                    output_1 = positional_out.eval(session=attention_head, feed_dict={inputs:word, input_pos:pos})
                    sent_vector.append(output_1)
                sent_vector = np.asarray(sent_vector).reshape((self.seq_len,self.emb_dim))
                output_2 = final_output_of_one_head.eval(session=attention_head, feed_dict={inputs_k:sent_vector, inputs_q:sent_vector, inputs_v:sent_vector})
                output_3 = res.eval(session=attention_head, feed_dict={inputs_final:output_2, inputs_for_norm:sent_vector})
                output_4 = res_final.eval(session=attention_head, feed_dict={inputs_for_ffn:output_3, inputs_for_norm_final:output_3})
                sentences.append(output_4)
            final_output.append(sentences)
            return np.asarray(final_output).reshape(len(final_output[0]), self.seq_len, self.emb_dim)

                

In [187]:
sess = tf.compat.v1.Session()
init_op = tf.compat.v1.global_variables_initializer()
sess.run(init_op)
obj = TransformerEncoder(len(vocab), 5, 128, 512, 1024)

In [188]:
X =[[0, 1, 2, 3, 4],
[5, 1, 2, 3, 6]]
output = obj.call(X)

In [190]:
output

array([[[ 177.64572  , -164.9207   ,   41.02332  , ...,   28.80564  ,
         -104.76409  ,   53.713436 ],
        [ 177.41264  , -166.20238  ,   41.256176 , ...,   28.025219 ,
         -103.48805  ,   54.316177 ],
        [ 179.73714  , -109.02429  ,  -80.95682  , ...,  -80.20761  ,
         -108.17632  ,    1.9724544],
        [ 177.80676  , -165.60052  ,   39.81535  , ...,   27.40807  ,
         -103.270294 ,   53.764374 ],
        [   8.146667 , -205.72935  ,   34.00499  , ...,    9.163757 ,
           10.055157 ,  -68.68278  ]],

       [[  69.70959  ,  -38.025944 ,  518.7505   , ..., -113.50997  ,
          -32.82434  ,  -96.92849  ],
        [ 180.44513  , -116.1046   ,  564.216    , ...,  -52.037415 ,
          -99.88502  ,   52.78704  ],
        [ 179.19827  , -116.02184  ,  567.8015   , ...,  -51.307636 ,
         -101.023834 ,   51.634747 ],
        [ 180.43875  , -116.37273  ,  557.4599   , ...,  -52.848827 ,
         -100.2059   ,   52.353092 ],
        [  37.222523 ,  -9

In [55]:
sess = tf.compat.v1.Session()
t1 = (tf.Variable([[1,2],[3,4],[5,6]]))
t2 = tf.Variable([1,2,3])
print(t1, t2)
t3 = tf.subtract(tf.transpose(t1),t2)
new = tf.transpose(t3)
init_op = tf.compat.v1.global_variables_initializer()
sess.run(init_op)

new.eval(session=sess)

<tf.Variable 'Variable_29:0' shape=(3, 2) dtype=int32> <tf.Variable 'Variable_30:0' shape=(3,) dtype=int32>


array([[0, 1],
       [1, 2],
       [2, 3]])

In [162]:
import pickle
with open('0', 'rb') as file:
    data = pickle.load(file)

In [13]:
data[0]['tokens']

<tf.Tensor 'Const_10:0' shape=(63878, 20) dtype=int32>

In [31]:
data[0]['tokens']

<tf.Tensor 'Const_10:0' shape=(63878, 20) dtype=int32>

In [163]:
with open('bert_vocab_uncased.txt', 'r') as f:
    vocab_ = f.read()
    vocab = {}
    for i in (vocab_.split()):
        if i not in vocab.keys():
            vocab[i] = len(vocab)
        

In [164]:
import json
with open('D:\Transformers Implementation\Language Model\Data\enwiki20201020\\00c2bfc7-57db-496e-9d5c-d62f8d8119e3.json', 'r') as f:
    data = json.load(f)

In [165]:
data[0]['text']

"M-137 was a state trunkline highway in the US state of Michigan that served as a spur route to the Interlochen Center for the Arts and Interlochen State Park. It started south of the park and ran north between two lakes in the area and through the community of Interlochen to US Highway 31 (US 31) in Grand Traverse County. The highway was first shown without a number label on maps in 1930 and labeled after an extension the next year. The highway's current routing was established in the 1950s. Jurisdiction of the roadway was transferred from the Michigan Department of Transportation (MDOT) to the Grand Traverse County Road Commission in June 2020, and the highway designation was decommissioned in the process; signage was removed by August 2020 to reflect the changeover. ==Route description== M-137 began at the southern end of Interlochen State Park at an intersection with Vagabond Lane. Farther south, the roadway continues toward Green Lake Airport as County Road 137 (CR 137), also know

In [166]:
list_of_sent = data[0]['text'].split('.')
new_data = []
for i in list_of_sent:
    t = []
    for k in i.split():
        if k in vocab.keys():
            t.append(vocab[k])
    new_data.append(t)
    break
    

In [167]:
new_data

[[2001,
  1037,
  2110,
  3307,
  1999,
  1996,
  2110,
  1997,
  2008,
  2366,
  2004,
  1037,
  12996,
  2799,
  2000,
  1996,
  2005,
  1996,
  1998]]

In [168]:
import random
mask_positions = []
target_values = []
weights = []
new_sequences = []
done_data = []
for m, i in (enumerate(new_data)):
    if len(i) > 5:
        t = []
        t_1 = []
        t_2 = []
        for k, j in enumerate(i):
            if k < 20:
                if random.random() <= 0.15:
                    if len(t) < 3:
                        t.append(k)
                        t_1.append(j)
                        t_2.append(1)
                        i[k] = vocab['[MASK]']
        weights.append(t_2 if len(t_2) == 3 else (
            t_2 + [0]*(3-len(t_2))))
        mask_positions.append(t if len(t) == 3 else (
            t+[0]*(3-len(t_2))))
        target_values.append(t_1 if len(t_1) == 3 else (
            t_1+[0]*(3-len(t_1))))
        new_sequences.append(
            i[:20]+[0]*(20-len(i[:20])))

done_data.append(({'tokens': new_sequences, 'mask_positions': mask_positions},
                                          target_values, weights))


In [169]:
new_sequences[0],mask_positions[0], target_values[0], weights[0]

([2001,
  1037,
  2110,
  3307,
  1999,
  1996,
  2110,
  103,
  2008,
  2366,
  2004,
  1037,
  12996,
  103,
  2000,
  1996,
  2005,
  1996,
  1998,
  0],
 [7, 13, 0],
 [1997, 2799, 0],
 [1, 1, 0])

In [60]:
np.asarray(done_data[0][0]['tokens']).shape

(20, 20)

In [32]:
input_data = data[0]['tokens']

In [170]:
import keras_nlp
te = TransformerEncoder(
        vocab_size=len(vocab),
        seq_len=20,
        emb_dim=128,
        linear_layers_hidden_dim = 128,
        ffn_hiiden_dim = 128)
encoded_tokens = te.call(done_data[0][0]['tokens'])
outputs = keras_nlp.layers.MLMHead(
             activation="softmax",vocabulary_size = len(vocab))

final_out = outputs(encoded_tokens, mask_positions=done_data[0][0]["mask_positions"])


In [111]:
encoded_seq.shape

(20, 20, 128)

In [171]:
init_op = tf.compat.v1.global_variables_initializer()
sess.run(init_op)
final_out.eval(session=sess)

RuntimeError: Attempted to use a closed Session.

In [113]:
outputs.shape

TensorShape([1, 3, 30522])

In [96]:
vocab['[MASK]']

103

In [117]:
a = np.asarray(done_data[0][1]).reshape((1, 3))

In [118]:
a[0]

array([2110, 1996,    0])

In [119]:
one_hot_output = tf.one_hot(done_data[0][1], depth=len(vocab))

In [71]:
one_hot_output

<tf.Tensor 'one_hot_66:0' shape=(20, 3, 30522) dtype=float32>

In [148]:
loss = tf.abs(tf.subtract(final_out,one_hot_output))

In [144]:
train = tf.compat.v1.train.GradientDescentOptimizer(1.0).minimize(loss, var_list=)

In [146]:
inverse_vocab = {v:k for k,v in vocab.items()}
correctly_classified = 0
for epoch in range(10):
    _,loss_val = sess.run([train,loss])
   
    # for k in loss_val[0]:
    #     print(tf.add_n(k).eval(session=sess))
    # print("\n\n")
        
def predict():
    enc_seq = te.call(done_data[0][0]['tokens'])
    f_out = outputs(enc_seq, mask_positions=done_data[0][0]["mask_positions"])
    for i in f_out.eval(session=sess)[0]:
        print(np.argmax(i))
# predict()

In [156]:
tf.compat.v1.reset_default_graph()

In [157]:
import tensorflow as tf
tf.compat.v1.trainable_variables()

[]

In [152]:
with tf.GradientTape() as tape:
  # Forward pass
    enc_seq = te.call(done_data[0][0]['tokens'])
    f_out = outputs(enc_seq, mask_positions=done_data[0][0]["mask_positions"])
    loss = tf.abs(tf.subtract(final_out,one_hot_output))
    grad = tape.gradient(loss, tf.compat.v1.trainable_variables())

KeyboardInterrupt: 